<a href="https://colab.research.google.com/github/GauravPatel89/EVA-Track3-Assignments/blob/master/Session13/Assignment13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

import matplotlib.pyplot as plt


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
tf.enable_eager_execution()

In [0]:
BATCH_SIZE =  128#@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE =  2e-5#@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS =  300#@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out, stride = 1):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3,strides=stride,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self,c_in, c_out, stride = 1):
    super().__init__()
    self.stride = stride
    self.c_in = c_in
    self.c_out = c_out
    self.conv1 = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3,strides=stride,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.conv2 = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3,strides=1,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn2 = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.resAdjust = tf.keras.layers.Conv2D(filters=c_out, kernel_size=1,strides=stride,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)

  def call(self, inputs):
    h = tf.nn.relu(self.bn1(self.conv1(inputs)))
    h = self.bn2(self.conv2(h))

    if self.stride != 1 or self.c_in != self.c_out:
      h = self.resAdjust(inputs) + h
    else:
      h = inputs + h
      

    h = tf.nn.relu(h)  
    return h

In [0]:
class ResNet18(tf.keras.Model):
  def __init__(self, c=64 ):
    super().__init__()
    
    self.init_conv_bn = ConvBN(c,stride = 1)
    self.blk1 = ResBlk(c, c, stride = 1)
    self.blk2 = ResBlk(c, c, stride = 1)
    self.blk3 = ResBlk(c, c*2, stride = 2)
    self.blk4 = ResBlk(c*2, c*2, stride = 1)
    self.blk5 = ResBlk(c*2, c*4, stride = 1)
    self.blk6 = ResBlk(c*4, c*4, stride = 1)
    self.blk7 = ResBlk(c*4, c*8, stride = 2)
    self.blk8 = ResBlk(c*8, c*8, stride = 1)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    

  def call(self, x, y):
    h = self.blk4(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.blk8(self.blk7(self.blk6(self.blk5(h))))
    h = self.linear(self.pool(h))
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)

y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

train_mean = np.array([0.4914, 0.4822, 0.4465])
train_std = np.array([0.2023, 0.1994, 0.2010])

normalize = lambda x: ((x - train_mean) / train_std).astype('float32')
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 6s 0us/step


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model = ResNet18()
batches_per_epoch = len_train//BATCH_SIZE + 1

#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
#lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE

opt = tf.train.MomentumOptimizer(LEARNING_RATE, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


In [12]:

t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
prevBestAcc = 0.0

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

    

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()

  modelPathName = '/content/drive/My Drive/TSAI/Session13/BestModelWeights.h5'

  currentTestAcc = (test_acc/len_test)   
  if(currentTestAcc > prevBestAcc):
    print('Accuracy improved from {} to {}. Saving model {}'.format(prevBestAcc,currentTestAcc,modelPathName))
    prevBestAcc = currentTestAcc
    model.save_weights(modelPathName)

  if(currentTestAcc >= 0.92):
    print('Accuracy of >= 93% achieved. Exiting model training.') 
    model.save_weights(modelPathName)
    break     

  print('epoch:', epoch+1, 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


Accuracy improved from 0.0 to 0.5542. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 1 train loss: 1.6490271910095216 train acc: 0.42158 val loss: 1.264627949333191 val acc: 0.5542 time: 225.62135577201843



Accuracy improved from 0.5542 to 0.663. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 2 train loss: 1.072804864730835 train acc: 0.62172 val loss: 0.9860416000366211 val acc: 0.663 time: 434.92307138442993



Accuracy improved from 0.663 to 0.751. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 3 train loss: 0.8146298149108887 train acc: 0.7147 val loss: 0.7367141494750976 val acc: 0.751 time: 643.2960157394409



Accuracy improved from 0.751 to 0.7817. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 4 train loss: 0.6479188116455078 train acc: 0.77608 val loss: 0.6573136394500733 val acc: 0.7817 time: 853.0239145755768



Accuracy improved from 0.7817 to 0.8052. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 5 train loss: 0.5515711067962646 train acc: 0.81078 val loss: 0.5953291705608368 val acc: 0.8052 time: 1062.6669325828552


...


Accuracy improved from 0.916 to 0.9165. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 50 train loss: 0.01726322807878256 train acc: 0.99456 val loss: 0.3760554735183716 val acc: 0.9165 time: 10469.92930674553



Accuracy improved from 0.9165 to 0.9195. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
epoch: 51 train loss: 0.01458461514890194 train acc: 0.99564 val loss: 0.35757532806396486 val acc: 0.9195 time: 10680.180894613266



epoch: 52 train loss: 0.014135107425749301 train acc: 0.99532 val loss: 0.3556968159675598 val acc: 0.9173 time: 10888.707478523254



epoch: 53 train loss: 0.01443634918436408 train acc: 0.99538 val loss: 0.3784969040393829 val acc: 0.9124 time: 11097.601499319077



Accuracy improved from 0.9195 to 0.9202. Saving model /content/drive/My Drive/TSAI/Session13/BestModelWeights.h5
Accuracy of >= 93% achieved. Exiting model training.
